In [1]:
import json
from data.preprocess import DataProcessor
import os
from data.custom_dataset import CustomDataset
from data.utils import get_dataset, get_tokenizer, get_dataloader
from model.model import Model
import torch
import pandas as pd
import numpy as np
import torch.optim as optim
import ast
import os
import itertools
import torch.utils.checkpoint
from torch.cuda.amp import GradScaler, autocast
from training.trainer import Trainer
import argparse
import ipdb



# device
USE_GPU = True
dtype = torch.float32
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
cpu = torch.device('cpu')
print(device)

/vol/bitbucket/mh1022/dl_cw_pyenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:


### for debugging use cpu
#device = cpu

config = {
#    "run_name": "Run_test_TLWAN"
    "run_name": "prova"
    ,"project_path": '/vol/bitbucket/mh1022/temporal-modelling-icd'
    ,"base_checkpoint": os.path.join("", "RoBERTa-base-PM-M3-Voc-hf")
    ,"num_attention_heads": 1
    ,"num_layers": 1
    ,"lr": 5e-5
    ,"max_chunks": 16
    ,"grad_accumulation": 16
    ,"use_positional_embeddings": True
    ,"use_reverse_positional_embeddings": True
    ,"priority_mode": "Last"
    ,"priority_idxs": [1]
    ,"use_document_embeddings": True
    ,"use_reverse_document_embeddings": True
    ,"use_category_embeddings": True
    ,"num_labels": 50
    ,"use_all_tokens": True
    ,"final_aggregation": "cls"
    ,"only_discharge_summary": False
    ,"patience_threshold": 3
    ,"max_epochs": 20
    ,"save_model": True
    ,"load_from_checkpoint": False
    ,"checkpoint_name": "Run_all_notes_last_second_transf"
}


# process and aggregate raw data
dp = DataProcessor(dataset_path="dataset", config=config)
notes_agg_df = dp.aggregate_data()
categories_mapping = dp.get_categories_mapping()

# get tokenizer
tokenizer = get_tokenizer(config["base_checkpoint"])

# Get training / validation / test
dataset_config = {
    "max_chunks" : config["max_chunks"],
    "priority_mode" : config["priority_mode"],
    "priority_idxs" : config["priority_idxs"],
    "categories_mapping": categories_mapping
}
training_set = get_dataset(notes_agg_df, "TRAIN", tokenizer = tokenizer, **dataset_config)
# print(f"categories mapping in the training set: {training_set.categories_mapping}")

training_generator = get_dataloader(training_set)

validation_set =  get_dataset(notes_agg_df, "VALIDATION", tokenizer = tokenizer, **dataset_config)
validation_generator = get_dataloader(validation_set)

test_set = get_dataset(notes_agg_df, "TEST", tokenizer = tokenizer, **dataset_config)
test_generator = get_dataloader(test_set)

# validation_set = get_dataset(notes_agg_df, "VALIDATION", tokenizer = tokenizer, **dataset_config)
# validation_generscator = get_dataloader(validation_set)

# test_set = get_dataset(notes_agg_df, "TEST", tokenizer = tokenizer, **dataset_config)
# test_generator = get_dataloader(test_set)
  

/vol/bitbucket/mh1022/dl_cw_pyenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
usage: ipykernel_launcher.py [-h] [-n NUM_CHUNKS] [-r RUN_NAME]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"1ceed4e6-97c1-4c50-98c3-8e50e70d2e55" --shell=9002 --transport="tcp" --iopub=9004 --f=/homes/mh1022/.local/share/jupyter/runtime/kernel-v2-1988237wBzNHBcnM1ux.json


SystemExit: 2

/vol/bitbucket/mh1022/dl_cw_pyenv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
